# This script demonstrates how to optimize a variational autoencoder with talos

In [1]:
# VAE
import vae_tools # Always import first to define if keras or tf.kreas should be used
import vae_tools.sanity
import vae_tools.viz
import vae_tools.callbacks
import vae_tools.sampling
vae_tools.sampling.set_seed(0)
from vae_tools.mmvae import MmVae, ReconstructionLoss
vae_tools.sanity.check()

# DL
import tensorflow as tf
import keras
from keras.layers import Input, Dense, Lambda, Layer
from keras.optimizers import Adam, Nadam, RMSprop
from keras.datasets import mnist

# Math
import numpy as np
from scipy.stats import norm

# Plot
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import astetik

# Hyperparameter search
from talos.model.normalizers import lr_normalizer
import talos as ta

Using TensorFlow backend.


python version:  3.5.2
keras version: 2.2.4
tensorflow version: 1.12.0
matplotlib uses:  module://ipykernel.pylab.backend_inline
Available GPUs ['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
# Get the MNIST digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [3]:
# Define the network and parameter structure

p = {'lr': [10., 1, 0.1],
     'intermediate_dim': [64, 128],
     'activation':['elu'],
     'batch_size': [64, 128],
     'epochs': [20, 40],
     'optimizer': [Adam, Nadam, RMSprop],
     'beta': [1.0],
     'reconstruction_loss_metrics': [ReconstructionLoss.MSE],
     'z_dim': [2, 4]}


img_rows, img_cols, img_chns = 28, 28, 1
original_dim = img_rows * img_cols * img_chns
def vae_model(x_train, y_train, x_val, y_val, params):
    vae_tools.sampling.set_seed(0)
    encoder = [[
        Input(shape=(original_dim,)),
        Dense(params['intermediate_dim'], activation=params['activation']),
        Dense(int(params['intermediate_dim']/2), activation=params['activation'])
    ]]

    decoder = [[
        Dense(int(params['intermediate_dim']/2), activation=params['activation']), 
        Dense(params['intermediate_dim'], activation=params['activation']),
        Dense(original_dim, activation='sigmoid')
    ]]

    vae_obj = MmVae(params['z_dim'], encoder, decoder, [original_dim], params['beta'],
                    reconstruction_loss_metrics = [params['reconstruction_loss_metrics']], name='Vae')

    vae_model = vae_obj.get_model()
    vae_model.compile(optimizer=params['optimizer'](lr_normalizer(params['lr'], params['optimizer'])), loss=None)
    losses_cb = vae_tools.callbacks.Losses(data = x_val)
    out = vae_model.fit(x_train, shuffle=True, epochs=params['epochs'], batch_size=params['batch_size'], verbose = 0, callbacks=[losses_cb])
    # Append the losses to the history
    for k in losses_cb.history.keys():
        out.history['_'.join(k.split('/')[0].split('_')[:-1])] = losses_cb.history[k]

    return out, vae_model


In [4]:
# Parameter search
h = ta.Scan(x=x_train, y=x_train, params=p,
            model=vae_model,
            dataset_name='mnist',
            experiment_no='1',
            x_val=x_test, y_val=x_test)

 12%|█▏        | 17/144 [26:52<4:31:15, 128.15s/it]

KeyboardInterrupt: 

In [ ]:
# access the summary details
print(h.details)
# Create the report
r = ta.Reporting(h)

In [ ]:
def sort_params(r, metric='losslayer_reconstruction_mse', ascending=True):
    ''' Sort table 'r' (pandas DataFrame) wrt. 'metric'
    '''
    return r.data.sort_values(metric, ascending=ascending).drop(None, axis=1)
out = sort_params(r)
out.head(10)

In [ ]:
# a regression plot for two dimensions 
#r.plot_regs(x='losslayer_reconstruction_mse', y='losslayer_distribution_gaussian_prior')
astetik.regs(out, x='losslayer_reconstruction_mse', y='losslayer_distribution_gaussian_prior', x_label='Rec.', y_label='D_KL')
# up to two dimensional kernel density estimator
astetik.kde(out, x='losslayer_reconstruction_mse', y='losslayer_distribution_gaussian_prior', x_label='Rec.', y_label='D_KL')
# a simple histogram
astetik.hist(out, 'loss',bins=50, x_label='loss')

astetik.corr(out, color_grades=5, style='astetik', annot = True)

# a four dimensional bar grid
astetik.bargrid(out, x='batch_size', y='loss', hue='lr', col='intermediate_dim',  col_wrap=3)